In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
pio.templates.default = "plotly_white"

In [2]:
df = pd.read_csv(r"C:\Users\Abhi\Downloads\rfm_data.csv")

In [3]:
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [5]:
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
df.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.000000,1000.00000,1000.000000
mean,5554.789000,513.67781,554071.398000
std,2605.014863,286.09870,264695.448814
min,1011.000000,12.13000,100096.000000
25%,3273.000000,257.12000,313152.000000
50%,5538.000000,523.56500,564671.500000
75%,7821.750000,759.86000,783052.250000
max,9991.000000,999.44000,999695.000000


In [8]:
df['Location'].unique()

array(['Tokyo', 'London', 'New York', 'Paris'], dtype=object)

In [9]:
product_location_counts = df.groupby(['Location','ProductInformation']).size().unstack()
product_location_counts

ProductInformation,Product A,Product B,Product C,Product D
Location,,,,
London,54,60,74,58
New York,53,59,66,69
Paris,50,58,65,56
Tokyo,68,68,72,70


# DATA MANUPULATION

# Let's calculate RFM values

In [10]:
from datetime import datetime

df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

# Calculating Recency

df['Recency'] = (datetime.now().date() - df['PurchaseDate'].dt.date).dt.days

# Calculating Frequency
frequency = df.groupby('CustomerID')['OrderID'].count().reset_index()
frequency.rename(columns={'OrderID' : 'Frequency'}, inplace = True)
df = df.merge(frequency , on= 'CustomerID', how = 'left')

# Calculating Monetory Value

monetory = df.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetory.rename(columns={'TransactionAmount' : 'Monetory'} , inplace = True)
df = df.merge(monetory , on = 'CustomerID', how= 'left')

In [11]:
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,Monetory
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,245,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,245,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,245,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,245,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,245,1,739.56
...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,185,1,759.62
996,6669,2023-06-10,941.50,Product C,987025,New York,185,1,941.50
997,8836,2023-06-10,545.36,Product C,512842,London,185,1,545.36
998,1440,2023-06-10,729.94,Product B,559753,Paris,185,1,729.94


# Let's calculate RFM scores

In [12]:
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)

frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency

monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
df['RecencyScore'] = pd.cut(df['Recency'], bins=5, labels=recency_scores)

df['FrequencyScore'] = pd.cut(df['Frequency'], bins=5, labels=frequency_scores)

df['MonetaryScore'] = pd.cut(df['Monetory'], bins=5, labels=monetary_scores)

In [13]:
df['RecencyScore']  =  df['RecencyScore'].astype(int)
df['FrequencyScore'] = df['FrequencyScore'].astype(int)
df['MonetoryScore'] =  df['MonetaryScore'].astype(int)

In [27]:
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,Monetory,RecencyScore,FrequencyScore,MonetoryScore,RFM_Score,Value Segment,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,245,1,943.31,1,1,2,4,Low-Value,Can't Lose them
1,2188,2023-04-11,463.70,Product A,176819,London,245,1,463.70,1,1,1,3,Low-Value,Already Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,245,1,80.28,1,1,1,3,Low-Value,Already Lost
3,2559,2023-04-11,221.29,Product A,239145,London,245,1,221.29,1,1,1,3,Low-Value,Already Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,245,1,739.56,1,1,2,4,Low-Value,Can't Lose them
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,185,1,759.62,5,1,2,8,High-Value,Potential Loyalists
996,6669,2023-06-10,941.50,Product C,987025,New York,185,1,941.50,5,1,2,8,High-Value,Potential Loyalists
997,8836,2023-06-10,545.36,Product C,512842,London,185,1,545.36,5,1,2,8,High-Value,Potential Loyalists
998,1440,2023-06-10,729.94,Product B,559753,Paris,185,1,729.94,5,1,2,8,High-Value,Potential Loyalists


In [29]:
#calculating RFM score by combining indivdual score
df['RFM_Score'] = (df['RecencyScore'].astype(int) +
                   df['FrequencyScore'].astype(int) +
                   df['MonetoryScore'].astype(int))

In [30]:
# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
df['Value Segment'] = pd.qcut(df['RFM_Score'], q=3, labels=segment_labels)

In [31]:
#
segment_counts = df['Value Segment'].value_counts().reset_index()

segment_counts.columns = ['Value Segment', 'Count']

pastel_colors = px.colors.qualitative.Pastel


fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count', 
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

In [32]:

df['RFM Customer Segments'] = ''


df.loc[df['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
df.loc[(df['RFM_Score'] >= 6) & (df['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
df.loc[(df['RFM_Score'] >= 5) & (df['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
df.loc[(df['RFM_Score'] >= 4) & (df['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose them"
df.loc[(df['RFM_Score'] >= 3) & (df['RFM_Score'] < 4), 'RFM Customer Segments'] = " Already Lost"


print(df[['CustomerID', 'RFM Customer Segments']])

     CustomerID RFM Customer Segments
0          8814       Can't Lose them
1          2188          Already Lost
2          4608          Already Lost
3          2559          Already Lost
4          9482       Can't Lose them
..          ...                   ...
995        2970   Potential Loyalists
996        6669   Potential Loyalists
997        8836   Potential Loyalists
998        1440   Potential Loyalists
999        4759   Potential Loyalists

[1000 rows x 2 columns]


In [34]:
segment_pro_counts = df.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_pro_counts = segment_pro_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_pro_counts, 
                                         path=['Value Segment', 'RFM Customer Segments'], 
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

In [36]:
champion_segment = df[df['RFM Customer Segments']== 'Champions']

fig = go.Figure()

fig.add_trace(go.Box(y= champion_segment['RecencyScore'], name= 'Recency' ))
fig.add_trace(go.Box(y= champion_segment['FrequencyScore'], name= 'Frequency'))
fig.add_trace(go.Box(y= champion_segment['MonetoryScore'], name= 'Monetary' ))
                     
                     
fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

In [47]:
champion_segment = df[df['RFM Customer Segments']== 'Potential Loyalists']

fig = go.Figure()

fig.add_trace(go.Box(y= champion_segment['RecencyScore'], name= 'Recency' ))
fig.add_trace(go.Box(y= champion_segment['FrequencyScore'], name= 'Frequency'))
fig.add_trace(go.Box(y= champion_segment['MonetoryScore'], name= 'Monetary' ))
                     
                     
fig.update_layout(title='Distribution of RFM Values within Potential Loyalists Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

In [51]:
correlation_matrix = champion_segment[['RecencyScore', 'FrequencyScore', 'MonetoryScore']].corr()


fig_heatmap = go.Figure(data=go.Heatmap( z=correlation_matrix.values, x=correlation_matrix.columns,
                                        y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [37]:
import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = df['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])
fig

In [45]:
segment_scores = df.groupby('RFM Customer Segments')['RecencyScore' , 'FrequencyScore' , 'MonetoryScore'].mean().reset_index()
 



fig5 = go.Figure()


fig5.add_trace(go.Bar(x=segment_scores ['RFM Customer Segments'], y=segment_scores['RecencyScore'],
                     name='Recency Score', marker_color='#3182bd'))


fig5.add_trace(go.Bar(x=segment_scores['RFM Customer Segments'], y=segment_scores['FrequencyScore'],
                     name='Frequency Score', marker_color='#6baed6'))


fig5.add_trace(go.Bar(x=segment_scores['RFM Customer Segments'], y=segment_scores['MonetoryScore'],
                     name='Monetary Score', marker_color='#9ecae1'))



fig5.update_layout(barmode='group', title='Comparison of RFM Segments Based on Recency , Frequency, Monetary value',
                  xaxis_title='RFM segments', yaxis_title='Score', showlegend = True)


C:\Users\Abhi\AppData\Local\Temp\ipykernel_2796\2347260318.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

